In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

print('files imported')

files imported


In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
   
    if row != [] and row[1] != "Not assigned":
     
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

df = pd.DataFrame(res, columns = ["Postalcode", "Borough", "Neighborhood"])
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [3]:
df["Postalcode"] = df["Postalcode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df = df.groupby(["Postalcode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [5]:
df.drop(df[df.Neighborhood == 'Not assigned'].index, inplace=True)
df.drop_duplicates('Postalcode', inplace=True)
df.reset_index(drop = True, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
print("Shape: ", df.shape)

Shape:  (103, 3)


In [7]:
import csv
from geopy.geocoders import Nominatim

In [8]:
!wget -q -O 'geospatialdata.csv' https://cocl.us/Geospatial_data

In [9]:
df_coordinates = pd.read_csv('geospatialdata.csv')
df_coordinates.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df_coordinates

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
df = pd.merge(df, df_coordinates, on='Postalcode')
df_1=df[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
df_1.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
!pip install folium
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011


In [13]:
df_2 = df_1[df_1['Borough'].str.contains("Toronto")].reset_index(drop=True)

map_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start=12)

for lat, lng, borough, neighborhood in zip(
        df_2['Latitude'], 
        df_2['Longitude'], 
        df_2['Borough'], 
        df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [14]:
from pandas.io.json import json_normalize

In [15]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=500
    CLIENT_ID = 'BPB504KL005T1EQQXSPEROUVF1LGKTCEUNETOSSSLDFIHBKP' # changed my Foursquare ID
    CLIENT_SECRET = 'WAXOLMZKBGYD0CT3BJH3W4RSC3FJRMEF3FYQ5OTFICPPKRGM' # changed Foursquare Secret
    VERSION = '20201001' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    print("done")
    return df

In [16]:
def get_venue_details(venue_id):
        
    CLIENT_ID = 'BPB504KL005T1EQQXSPEROUVF1LGKTCEUNETOSSSLDFIHBKP' 
    CLIENT_SECRET = 'WAXOLMZKBGYD0CT3BJH3W4RSC3FJRMEF3FYQ5OTFICPPKRGM' 
    VERSION = '20201001' # Foursquare API version
    
    
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df
     

In [17]:
column_names=['Borough', 'Neighborhood', 'ID','Name']
indian_rest_to=pd.DataFrame(columns=column_names)
count=1
for row in df_2.values.tolist():
    Postalcode,Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    Indian_restaurants=venues[venues['Category']=='Indian Restaurant']   
    print('(',count,'/',len(df_2),')','Indian Restaurants in '+Neighborhood+', '+Borough+':'+str(len(Indian_restaurants)))
    for restaurant_detail in Indian_restaurants.values.tolist():
        id, name , category=restaurant_detail
        indian_rest_to = indian_rest_to.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

done
( 1 / 39 ) Indian Restaurants in The Beaches, East Toronto:2
done
( 2 / 39 ) Indian Restaurants in The Danforth West, Riverdale, East Toronto:1
done
( 3 / 39 ) Indian Restaurants in India Bazaar, The Beaches West, East Toronto:8
done
( 4 / 39 ) Indian Restaurants in Studio District, East Toronto:0
done
( 5 / 39 ) Indian Restaurants in Lawrence Park, Central Toronto:0
done
( 6 / 39 ) Indian Restaurants in Davisville North, Central Toronto:0
done
( 7 / 39 ) Indian Restaurants in North Toronto West,  Lawrence Park, Central Toronto:0
done
( 8 / 39 ) Indian Restaurants in Davisville, Central Toronto:3
done
( 9 / 39 ) Indian Restaurants in Moore Park, Summerhill East, Central Toronto:0
done
( 10 / 39 ) Indian Restaurants in Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park, Central Toronto:0
done
( 11 / 39 ) Indian Restaurants in Rosedale, Downtown Toronto:0
done
( 12 / 39 ) Indian Restaurants in St. James Town, Cabbagetown, Downtown Toronto:1
done
( 13 / 39 ) Indian Res

In [18]:
indian_rest_to.head()

,Borough,Neighborhood,ID,Name
0,East Toronto,The Beaches,4dcd7c6352b1f8915b7e7f7e,Delhi Bistro
1,East Toronto,The Beaches,4b60e4a1f964a52023002ae3,ali's tandoori curry house
2,East Toronto,"The Danforth West, Riverdale",4c1d5337eac020a1cb1048c2,Sher-E-Punjab
3,East Toronto,"India Bazaar, The Beaches West",4ae0c7a8f964a520638221e3,Udupi Palace
4,East Toronto,"India Bazaar, The Beaches West",4afc9816f964a520312422e3,Motimahal


In [19]:
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
indian_rest_stats_to=pd.DataFrame(columns=column_names)
count=1


for row in indian_rest_to.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
      
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(indian_rest_to),')','processed')
    indian_rest_stats_to = indian_rest_stats_to.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

                         ID          Name  Likes  Rating  Tips
0  4dcd7c6352b1f8915b7e7f7e  Delhi Bistro      6     6.9     3
( 1 / 27 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b60e4a1f964a52023002ae3
( 2 / 27 ) processed
                         ID           Name  Likes  Rating  Tips
0  4c1d5337eac020a1cb1048c2  Sher-E-Punjab     10     7.3     8
( 3 / 27 ) processed
                         ID          Name  Likes  Rating  Tips
0  4ae0c7a8f964a520638221e3  Udupi Palace     78     8.3    31
( 4 / 27 ) processed
                         ID       Name  Likes  Rating  Tips
0  4afc9816f964a520312422e3  Motimahal     24     8.0    13
( 5 / 27 ) processed
                         ID              Name  Likes  Rating  Tips
0  4bac30a2f964a52018ea3ae3  Bombay Chowpatty      7     7.4     5
( 6 / 27 ) processed
                         ID                Name  Likes  Rating  Tips
0  4ad9052cf964a520301721e3  Regency Restaurant      

KeyError: 'venue'

In [ ]:
indian_rest_stats_to.head()

In [ ]:
indian_rest_stats_to.shape

In [ ]:
indian_rest_stats_to['Likes']=indian_rest_stats_to['Likes'].astype('float64')
indian_rest_stats_to['Tips']=indian_rest_stats_to['Tips'].astype('float64')
indian_rest_stats_to.info()

In [ ]:
indian_rest_stats_to.iloc[indian_rest_stats_to['Likes'].idxmax()]

In [ ]:
indian_rest_stats_to.iloc[indian_rest_stats_to['Rating'].idxmax()]

In [ ]:
to_neighborhood_stats=indian_rest_stats_to.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
to_neighborhood_stats.columns=['Neighborhood','Average Rating']

In [ ]:
to_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

In [ ]:
to_borough_stats=indian_rest_stats_to.groupby('Borough',as_index=False).mean()[['Borough','Rating']]
to_borough_stats.columns=['Borough','Average Rating']
to_borough_stats.sort_values(['Average Rating'],ascending=False).head()

In [ ]:
to_onehot=pd.get_dummies(indian_rest_to[['Name']],prefix="",prefix_sep="")

to_onehot['Neighborhood']=indian_rest_to['Neighborhood']

fixed_columns=[to_onehot.columns[-1]]+list(to_onehot.columns[:-1])
to_onehot=to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head(27)

In [ ]:
indian_grouped = to_onehot.groupby('Neighborhood').mean().reset_index()
indian_grouped.head(7)

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = indian_grouped['Neighborhood']

for ind in np.arange(indian_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(indian_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
from sklearn.cluster import KMeans

to_data = df_1.drop(16)
kclusters = 5

indian_grouped_clustering = indian_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(indian_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]